<a href="https://colab.research.google.com/github/yaelecker/EngGer-Model/blob/main/prepared_goalType_training_dutch_KU_LEUVEN_and_NFI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#%%capture
!pip install -qU transformers sentence-transformers
!pip install pytorch-ranger
!pip install audtorch
!pip install tqdm
!pip install datasets

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch # Torch is the machine learning framework originated from Facebook. Provides the computations for the neural network - looking at how much each cell contributed to the error in order to change it.
from sentence_transformers import SentenceTransformer, util, models, SentencesDataset, losses, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch import nn, optim
from torch.utils.data import DataLoader
import csv
import numpy as np # awesome math library
from os import walk
import pandas as pd # dataframes
import ipywidgets as widgets
import io
from pytorch_ranger import Ranger
from tqdm.auto import tqdm
import os
import time
timestr = time.strftime("%Y-%m-%d")


In [ ]:
seed = 123
torch.manual_seed(seed)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/goaltype/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import math

def round_to_first_non_zero_digit(x):
  if x == 0:
      return 0  # Early exit for zero, as it has no non-zero digits
  # Find the order of magnitude of the number
  order_of_magnitude = math.floor(math.log10(abs(x))) - 1
  # Scale the number so the first non-zero digit is to the left of the decimal point
  scaled_number = x / (10 ** order_of_magnitude)
  # Round the scaled number to one significant figure
  rounded_scaled_number = round(scaled_number, 0)
  # Scale back to the original order of magnitude
  return rounded_scaled_number * (10 ** order_of_magnitude)

In [ ]:
from torch.utils.data import DataLoader, Dataset


def add_prototypes_sentence_data(df, weight = 3):
  MANT_SENTENCE = ["We aim to invest in maintaining the current state", "We willen investeren in het behoud van de huidige staat", "Wir wollen in die Erhaltung des derzeitigen Zustands investieren"]
  PROG_SENTENCE = ["We aim to invest in making changes and improving the current state", "We willen investeren in het aanbrengen van veranderingen en het verbeteren van de huidige staat", "Wir wollen in Veränderungen und die Verbesserung des aktuellen Zustands investieren"]
  PROT_SENTENCE = ["We aim to invest in preventing negative changes to the current state","We willen investeren in het voorkomen van negatieve veranderingen in de huidige staat","Wir wollen in die Verhinderung negativer Veränderungen des derzeitigen Zustands investieren"]
  df_mant = pd.DataFrame({'item': MANT_SENTENCE,
                         'prog_human': [0.0,0.0,0.0],
                         'mant_human': [1.0,1.0,1.0],
                         'prot_human': [0.0,0.0,0.0],
                         'is_prototype' : [True, True, True],
                         'weight': [weight, weight, weight]})

  df_prog = pd.DataFrame({'item': PROG_SENTENCE,
                         'prog_human': [1.0,1.0,1.0],
                         'mant_human': [0.0,0.0,0.0],
                         'prot_human': [0.0,0.0,0.0],
                         'is_prototype' : [True, True, True],
                         'weight': [weight, weight, weight]})
  df_prot = pd.DataFrame({'item': PROT_SENTENCE,
                         'prog_human': [0.0,0.0,0.0],
                         'mant_human': [0.0,0.0,0.0],
                         'prot_human': [1.0,1.0,1.0],
                         'is_prototype' : [True, True, True],
                         'weight': [weight, weight, weight]})

  return pd.concat((df_mant,df_prog,df_prot, df))

class PretrainDataset(Dataset):
  def __init__(self, dataframe):
    dataframe['is_prototype'].fillna(False,inplace=True)
    self.items = dataframe[['item','prog_human', 'mant_human', 'prot_human']]
    prototypes = dataframe[dataframe['is_prototype']][['item','prog_human', 'mant_human', 'prot_human']]
    prototypes = prototypes.rename(columns={'item': 'prototype', 'prog_human': 'prog_proto', 'mant_human': 'mant_proto', 'prot_human': 'prot_proto'})
    def compute_distance(row):
      if row['prog_proto'] == 1.0:
        return row['prog_human']
      if row['mant_proto'] == 1.0:
        return row['mant_human']
      if row['prot_proto'] == 1.0:
        return row['prot_human']
      else:
        return 0.0

    self.items = self.items.merge(prototypes, how='cross')
    mask = self.items['item'] != self.items['prototype']
    self.items = self.items[mask]
    self.items['distance'] = self.items.apply(lambda x: 0.1 + 0.8 * compute_distance(x), axis=1).astype('float32')
    self.items = self.items[self.items['distance'].notna()]



  def __len__(self):
    return len(self.items)

  def __getitem__(self, index):
    # Retrieve data at the specified index
    self.last_idx=index
    subset = self.items.iloc[index]
    if (type(subset) is pd.Series):
      return InputExample(texts=[subset['item'], subset['prototype']], label=subset['distance'].astype('float32'))
    return [InputExample(texts=[x['item'], x['prototype']], label=x['distance']) for x in subset.iterrows()]

class TextDataset(Dataset):
    def __init__(self, dataframe, prompt = None, add_prototypes=False, paraphrases_column=None):
        """
        Args:
            dataframe (pd.DataFrame): DataFrame containing the sentences and their labels.
        """
        self.last_idx = None
        self.dataframe = dataframe
        self.prompt = prompt

        group_sizes = self.dataframe.groupby('source')['source'].transform('size')
        total_rows = len(self.dataframe)

        # Calculate weight
        self.dataframe['weight'] = total_rows / group_sizes
        # Scale weights so the minimum weight is 1
        min_weight = self.dataframe['weight'].min()
        self.dataframe['weight'] = self.dataframe['weight'] / min_weight

        if paraphrases_column != None:
          exploded_df = self.dataframe.explode(paraphrases_column)
          exploded_df['item'] = exploded_df[paraphrases_column]
          self.dataframe = exploded_df


        if add_prototypes:
          max_weight = self.dataframe['weight'].max()
          self.dataframe = add_prototypes_sentence_data(self.dataframe, 3 * max_weight)


    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        # Retrieve data at the specified index
        sentence = self.dataframe.iloc[index]['item']
        labels = self.dataframe.iloc[index][['prog_human', 'mant_human', 'prot_human']].values.astype(float)
        labels = torch.tensor(labels, dtype=torch.float)
        weights = self.dataframe.iloc[index][['weight']].values.astype(float)
        weights = torch.tensor(weights, dtype=torch.float)
        weights = weights.expand(3)
        #print(f"Index: {index}, Data shape: {len(sentence), labels.shape, weights.shape}")
        self.last_idx=index
        if self.prompt != None:
          return f"{self.prompt}: {sentence}", labels, weights
        else:
          return sentence, labels, weights


def read_file(filename, frac, random=None, filter=None):
  if random is None:
    random = seed
  df = pd.read_csv(filename)
  if filter is not None:
    df = df[df.apply(filter,axis=1)]
  train_dfs = []
  test_dfs = []
  for s in df['source'].unique():
    dfc = df.loc[df['source'] == s]
    train = dfc.sample(frac=frac,random_state=random)
    test = dfc.drop(train.index)
    train_dfs.append(train)
    test_dfs.append(test)

  return pd.concat(train_dfs), pd.concat(test_dfs)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


def correlations(df):
  return pd.DataFrame({'corr_prog': [df['prog_human'].corr(df['prog_ai'])], 'corr_mant': [df['mant_human'].corr(df['mant_ai'])], 'corr_prot': [df['prot_human'].corr(df['prot_ai'])]})

def hit_counts(df,threshold=0.05):
  df.loc[:,'d_prog'] = (df['prog_ai'] - df['prog_human']).abs()
  df.loc[:,'d_mant'] = (df['mant_ai'] - df['mant_human']).abs()
  df.loc[:,'d_prot'] = (df['prot_ai'] - df['prot_human']).abs()
  num_rows = len(df.index)
  #print(df['d_prog'].lt(threshold).sum(), df['d_prog'])
  return pd.DataFrame({'hp_prog': [df['d_prog'].lt(threshold).sum()/num_rows], 'hp_mant': [df['d_mant'].lt(threshold).sum()/num_rows], 'hp_prot': [df['d_prot'].lt(threshold).sum()/num_rows]})

def all_measurements(df, f):
  dfs = []
  all = f(df)
  all['source'] = 'combined'
  dfs.append(all)
  for s in df.source.unique():
    dfc = f(df.loc[df['source'] == s])
    dfc['source'] = s
    dfs.append(dfc)
  return pd.concat(dfs)


def all_correlations(df):
  return all_measurements(df, correlations)

def all_hits_counts(df, t):
  return all_measurements(df, lambda x: hit_counts(x,t))

class ParallelLayer(nn.Module):
  def __init__(self,
              modules
              ):
    super(ParallelLayer, self).__init__()
    self.elements = nn.ModuleList(modules)

  def forward(self, input):
    tmp_emb = []
    for m in self.elements:
      emb = m(input)
      tmp_emb.append(emb)
    if (len(tmp_emb)>1):
      return torch.cat(tmp_emb, dim=1)
    else:
      return tmp_emb[0]

  def __iter__(self):
    return iter(self.elements)

class EmbeddingEncoder(nn.Module):
  def __init__(self,
              embedder, trainable=True):
    super(EmbeddingEncoder, self).__init__()
    self.embedder = embedder
    if trainable:
      for param in self.embedder.parameters():
        param.requires_grad = True

  def forward(self, input):
    return self.embedder.encode(input, convert_to_tensor=True)

  def get_sentence_embedding_dimension(self):
    return self.embedder.get_sentence_embedding_dimension()



class ConcatenatingSkipModule(nn.Module):
  def __init__(self,
            modules, trainable=True):
    super(ConcatenatingSkipModule, self).__init__()
    self.elements = nn.ModuleList(modules)

  def forward(self, x):
    prev = None
    for h in self.elements:
      if prev == None:
        x = h(x)
      else:
        inp = torch.cat((prev,x),dim=1)
        prev = x
        x = h( inp )
    final = torch.cat((prev,x),dim=1)

import torch
import torch.nn as nn
import torch.nn.functional as F

class SimulatedSelfAttention(nn.Module):
  def __init__(self, input_dim, heads=1):
    super(SimulatedSelfAttention, self).__init__()
    self.heads = heads
    self.input_dim = input_dim

    # Typically, dimension should be divisible by the number of heads
    self.query = nn.Linear(input_dim, input_dim, bias=False)
    self.key = nn.Linear(input_dim, input_dim, bias=False)
    self.value = nn.Linear(input_dim, input_dim, bias=False)

  def forward(self, x):
    # Simulate a sequence by expanding the single vector
    batch_size = x.shape[0]
    x_expanded = x.unsqueeze(1).expand(-1, self.heads, -1)  # Expand to simulate 'heads' number of sequences

    # Apply self-attention
    queries = self.query(x_expanded)
    keys = self.key(x_expanded)
    values = self.value(x_expanded)

    # Scaled dot-product attention
    attention_scores = torch.matmul(queries, keys.transpose(-2, -1)) / self.input_dim ** 0.5
    attention = F.softmax(attention_scores, dim=-1)
    attended = torch.matmul(attention, values).sum(1)  # Combine heads

    return attended

class FixedQueryAttention(nn.Module):
  def __init__(self, input_dim, query_vector, heads=1):
    super(FixedQueryAttention, self).__init__()
    self.heads = heads
    self.input_dim = input_dim
    self.register_buffer('y', query_vector.unsqueeze(0))
    self.y = query_vector

    # Typically, dimension should be divisible by the number of heads
    self.query = nn.Linear(input_dim, input_dim, bias=False)
    self.key = nn.Linear(input_dim, input_dim, bias=False)
    self.value = nn.Linear(input_dim, input_dim, bias=False)

  def forward(self, x):
    # Simulate a sequence by expanding the single vector
    batch_size = x.shape[0]
    x_expanded = x.unsqueeze(1).expand(-1, self.heads, -1)  # Expand to simulate 'heads' number of sequences

    # Prepare the fixed query vector, expanding it for compatibility with batch and head dimensions
    y_expanded = self.y.expand(batch_size, self.heads, -1)  # Ensure y is compatible with x in batch and head dimensions

    queries = self.query(y_expanded)
    keys = self.key(x_expanded)
    values = self.value(x_expanded)

    # Scaled dot-product attention
    attention_scores = torch.matmul(queries, keys.transpose(-2, -1)) / self.input_dim ** 0.5
    attention = F.softmax(attention_scores, dim=-1)
    attended = torch.matmul(attention, values).sum(1)  # Combine heads

    return attended

class MultiQueryFixedAttention(nn.Module):
    def __init__(self, input_dim, query_vectors, heads=1):
        super(MultiQueryFixedAttention, self).__init__()
        self.heads = heads
        self.input_dim = input_dim
        self.num_queries = query_vectors.shape[0]

        # Store query vectors as a registered buffer
        self.register_buffer('query_vectors', query_vectors.unsqueeze(1))  # Shape: [num_queries, 1, input_dim]

        # Define the linear transformations for queries, keys, and values
        self.query = nn.Linear(input_dim, input_dim, bias=False)
        self.key = nn.Linear(input_dim, input_dim, bias=False)
        self.value = nn.Linear(input_dim, input_dim, bias=False)

    def forward(self, x):
        batch_size = x.shape[0]

        # Expand x to simulate 'heads' number of sequences and repeat it for each query
        x_expanded = x.unsqueeze(1).expand(-1, self.heads, -1)  # Shape: [batch_size, heads, input_dim]
        x_expanded = x_expanded.unsqueeze(2).repeat(1, 1, self.num_queries, 1)  # Shape: [batch_size, heads, num_queries, input_dim]

        # Prepare the fixed query vectors, expanding them for compatibility with batch, head, and sequence dimensions
        query_vectors_expanded = self.query_vectors.expand(self.num_queries, self.heads, self.input_dim)
        query_vectors_expanded = query_vectors_expanded.repeat(batch_size, 1, 1, 1)  # Shape: [batch_size, heads, num_queries, input_dim]

        # Apply linear transformations
        queries = self.query(query_vectors_expanded)
        keys = self.key(x_expanded)
        values = self.value(x_expanded)

        # Scaled dot-product attention
        attention_scores = torch.matmul(queries, keys.transpose(-2, -1)) / (self.input_dim ** 0.5)
        attention = F.softmax(attention_scores, dim=-1)
        attended = torch.matmul(attention, values).mean(dim=1)#.squeeze(1)  # Average combine heads, reduce over head dimension
        #print(attended.shape)
        return attended.view(-1, self.input_dim * self.heads)


class LearnedAttention(nn.Module):
  def __init__(self, input_dim,  heads=1):
    super(LearnedAttention, self).__init__()
    self.heads = heads
    self.input_dim = input_dim
    self.y = nn.Parameter(torch.randn(input_dim))

    # Typically, dimension should be divisible by the number of heads
    self.query = nn.Linear(input_dim, input_dim, bias=False)
    self.key = nn.Linear(input_dim, input_dim, bias=False)
    self.value = nn.Linear(input_dim, input_dim, bias=False)

  def forward(self, x):
    # Simulate a sequence by expanding the single vector
    batch_size = x.shape[0]
    x_expanded = x.unsqueeze(1).expand(-1, self.heads, -1)  # Expand to simulate 'heads' number of sequences

    # Prepare the fixed query vector, expanding it for compatibility with batch and head dimensions
    y_expanded = self.y.expand(batch_size, self.heads, -1)  # Ensure y is compatible with x in batch and head dimensions

    queries = self.query(y_expanded)
    keys = self.key(x_expanded)
    values = self.value(x_expanded)

    # Scaled dot-product attention
    attention_scores = torch.matmul(queries, keys.transpose(-2, -1)) / self.input_dim ** 0.5
    attention = F.softmax(attention_scores, dim=-1)
    attended = torch.matmul(attention, values).sum(1)  # Combine heads

    return attended

class GoalTypeScoringModel(nn.Module):
    def __init__(self,
                 pretrained_models,
                 dropout_rate=0.2,
                 hidden=[32,16],
                 activation=nn.ReLU,
                 enable_skip=False,
                 attend = 0,
                 trainable_embeddings = False,
                 batch_norm=False,
                 query=None
                 ):
        super(GoalTypeScoringModel, self).__init__()
        layers = []
        # Assume we want to fine-tune, so keep the original state of model
        embeddings = ParallelLayer([EmbeddingEncoder(p, trainable=trainable_embeddings) for p in pretrained_models])
        layers.append(embeddings)
        self.enable_skip = enable_skip

        classifier_size=sum([e.get_sentence_embedding_dimension() for e in embeddings]) #self.sentence_transformer.get_sentence_embedding_dimension()
        if attend > 0:
          if query != None:
            with torch.no_grad():
              if type(query) is type([]):
                query_vectors = embeddings(query)
                attention_layer = MultiQueryFixedAttention(input_dim=classifier_size, query_vectors=query_vectors, heads=attend)
                classifier_size = classifier_size * attend
              else:
                query_vector = embeddings([query])
                attention_layer = FixedQueryAttention(input_dim=classifier_size, query_vector=query_vector, heads=attend)
          else:
            attention_layer = SimulatedSelfAttention(input_dim=classifier_size, heads=attend)
          layers.append(nn.Dropout(dropout_rate))
          layers.append(attention_layer)


        if batch_norm:
          def make_hidden_layer(in_dim, out_dim, dropout_rate):
            return nn.Sequential(nn.Dropout(dropout_rate),nn.Linear(in_dim, out_dim),nn.BatchNorm1d(out_dim),activation())
        else:
          def make_hidden_layer(in_dim, out_dim, dropout_rate):
            return nn.Sequential(nn.Dropout(dropout_rate),nn.Linear(in_dim, out_dim),activation())

        if len(hidden) >= 1:
          hidden.insert(0, classifier_size)
          if self.enable_skip:
            pairs_list = [(sum(hidden[i:i+1]), hidden[i + 1]) for i in range(len(hidden) - 1)]
            classifier_size = sum(hidden)
            hidden_layer = nn.ConcatenatingSkipModule([  make_hidden_layer(x,y,dropout_rate) for x,y in pairs_list])
          else:
            pairs_list = [(hidden[i], hidden[i + 1]) for i in range(len(hidden) - 1)]
            classifier_size = hidden[-1]
            hidden_layer = nn.Sequential(*[  make_hidden_layer(x,y,dropout_rate) for x,y in pairs_list])
          layers.append(hidden_layer)
        layers.append(nn.Dropout(dropout_rate))
        classifier = nn.Linear(classifier_size, 3)
        layers.append(classifier)

        self.all = nn.Sequential(*layers)

    def forward(self, sentences, hidden = None):
      """
        tmp_emb = []
        for e in self.sentence_transformers:
          emb = e.encode(sentences, convert_to_tensor=True)
          tmp_emb.append(emb)

        # Embeddings now require gradient
        if (len(tmp_emb)>1):
          embedded = torch.cat(tmp_emb, dim=1)
        else:
          embedded = tmp_emb[0]
        #embedded = self.sentence_transformer.encode(sentences, convert_to_tensor=True)
        outs = []
        x = self.dropout(embedded)
        if (self.enable_skip):
          outs.append(embedded)
          if self.hidden != None:
            for h in self.hidden:
              if x == None:
                x = h(embedded)
              else:
                x = h( torch.cat(outs,dim=1))
              outs.append(x)
          final = torch.cat(outs,dim=1)
        else:
          if self.hidden != None:
            final = self.hidden(x)
          else:
            final = x
        x = self.classifier(final)
        return torch.sigmoid(x)
      """
      x = self.all(sentences)
      return torch.sigmoid(x)


def evaluate_model(model, data_loader, loss_fn, device, t=0.05):
    model.to(device)
    model.eval()  # Put the model in evaluation mode
    total_loss = 0
    total_samples = 0
    with torch.no_grad():  # Disable gradient computation
      for sentences, labels, weights in data_loader:
        labels = labels.to(device)
        weights = weights.to(device).expand(-1, 3)
        outputs = model(sentences)
        loss = loss_fn(outputs, labels, weights)
        total_loss += loss.item() * labels.size(0)
        total_samples += labels.size(0)
        #dists = torch.abs(outputs - labels) <= threshold
        #all_dists += dists.float().sum()

      df = data_loader.dataset.dataframe
      df['predictions'] = df['item'].apply(lambda x: predict(model, x, device))

      # Convert predictions list of arrays into separate columns
    predictions_df = pd.DataFrame(df['predictions'].tolist(), index= df.index, columns=['prog_ai', 'mant_ai', 'prot_ai'])
    df = pd.concat([df, predictions_df], axis=1).drop(columns=['predictions'])

    average_loss = total_loss / total_samples

    return average_loss,all_correlations(df)

def predict(model, sentence, device):
    model.to(device)
    model.eval()  # Ensure the model is in evaluation mode
    with torch.no_grad():  # Disable gradient computation
      # You might need to adjust this part depending on how your model expects input
      output = model([sentence])  # Assume the model can handle list input and convert to tensor internally
      predicted_rating = output.cpu().numpy().squeeze(0)  # Convert to NumPy array for easier manipulation
    return predicted_rating

In [ ]:
from typing import List, Set, Dict, Tuple,Callable
from dataclasses import dataclass, field

@dataclass
class ModelSpec():
    embedding: List[str] = field(default_factory=lambda: ['BAAI/bge-m3'])
    dropout_rate : float = 0.5
    hidden_layers : List[int] = field(default_factory=lambda: [16, 8])
    activation :Callable =  nn.LeakyReLU
    batch_size :int =  2
    optimizer: Callable = Ranger
    learning_rate : float =  0.001
    epochs : int = 10
    pretrain_epochs: int = 2
    pretrain_batch_size: int = 1
    attention_heads : int = 0
    trainable_embeddings :bool = False
    batch_norm :bool = False
    query : str = None
    model : GoalTypeScoringModel = None
    best_validation_loss: float=float('inf')
    best_epoch: int=-1

In [ ]:


def save_checkpoint(state, filename="checkpoint.pth.tar"):
  torch.save(state, filename)



class GoalTypeScoringModelFactory():

  def __init__(self, specs:List[ModelSpec], name:str="goal_type_model"):
    self.specs = specs
    self.name=name

  def evaluate_all(self, data_loader, device=None):
    if device is None:
      device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    for i, spec in enumerate(self.specs):
      print(f"evaluating model {i}")
      model = spec['model']
      model.to(device)
      model.eval()  # Put the model in evaluation mode
      with torch.no_grad():  # Disable gradient computation
        test_loss,corr = evaluate_model(model, data_loader, loss_fn, device, 0.1)
        print(f"Average Test Loss: {test_loss}")
        print("correlations:")
        print(corr)
  def evaluate_avg(self, data_loader, device=None):
    if device is None:
      device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    total_loss = 0
    total_samples = 0

    for i, spec in enumerate(self.specs):
      model = spec['model']
      model.to(device)
      model.eval()  # Put the model in evaluation mode
      with torch.no_grad():  # Disable gradient computation
        for sentences, labels in data_loader:
          labels = labels.to(device)
          outputs = model(sentences)
          loss = loss_fn(outputs, labels)
          total_loss += loss.item() * labels.size(0)
          total_samples += labels.size(0)
          #dists = torch.abs(outputs - labels) <= threshold
          #all_dists += dists.float().sum()

        df = data_loader.dataset.dataframe
        df[f'predictions_{i}'] = df['item'].apply(lambda x: predict(model, x, device))
      model.to("cpu")
      predictions_df = pd.DataFrame(df[f'predictions_{i}'].tolist(), index= df.index, columns=[f'prog_{i}', f'mant_{i}', f'prot_{i}'])
      df = pd.concat([df, predictions_df], axis=1).drop(columns=[f'predictions_{i}'])
      data_loader.dataset.dataframe = df
    average_loss = total_loss / total_samples

    df = data_loader.dataset.dataframe

    for goal_type in ['prog', 'mant', 'prot']:
      pred_cols = [f'{goal_type}_{i}' for i in range(len(self.specs))]
      df[f'{goal_type}_ai'] = df[pred_cols].mean(axis=1)
      df.drop(pred_cols, axis=1, inplace=True)

    return average_loss,all_correlations(df)


  def predict(self, sentence):
    predictions = []
    for spec in self.specs:
      model = spec.model

      model.eval()  # Ensure the model is in evaluation mode
      with torch.no_grad():  # Disable gradient computation
        # You might need to adjust this part depending on how your model expects input
        output = self.model([sentence])  # Assume the model can handle list input and convert to tensor internally
        predicted_rating = output.cpu().numpy().squeeze(0)  # Convert to NumPy array for easier manipulation
        predictions.append(predicted_rating)
      model.to("cpu")
    return sum(predictions)/len(predictions)

  def load_all(self):
    for s_num, s in enumerate(self.specs):
      print(f"training model for spec {s}")
      embedding = s.get('embedding', 'BAAI/bge-m3')
      dropout_rate = s.get('dropout_rate', 0.5)
      hidden_layers = s.get('hidden_layers', [16, 8])
      activation = s.get('activation', nn.LeakyReLU)
      batch_size= s.get('batch_size', 2)
      optimizer_f = s.get('optimizer', Ranger)
      learning_rate = s.get('learning_rate', 0.001)
      epochs = s.get('epochs', 10)
      loss_fn_f = s.get('loss_fn', F.mse_loss)
      sample_frac = s.get('sample_frac', 1.0)
      attend = s.get('attend', None)


      print("loading best checkpoint")
      checkpoint = torch.load(f'{self.name}.{s_num}.pth.tar', map_location='cpu')
      model.load_state_dict(checkpoint['state_dict'])
      optimizer.load_state_dict(checkpoint['optimizer'])
      model.to("cpu")
      s['model'] = model


  def train_all(self, train_set, test_set, device=None, pretrain=True):
    if device is None:
      device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    for s_num, s in enumerate(self.specs):

      if s.batch_norm and s.batch_size < 20:
        print(f"batch size {s.batch_size} too small for batch norm. turning off batch norm")
        s.batch_norm = False

      print(f"training model for spec\n\t {s}")

      def loss_fn(output, target, weights):
        # Create a mask that is 1 where target is not NaN, zero otherwise
        mask = ~torch.isnan(target)
        # Apply mask to only compute loss on non-NaN elements
        # Check if mask needs to be adjusted to match the output dimensions
        if mask.dim() < output.dim():
            mask = mask.unsqueeze(-1)  # Add dimension at the end if necessary
        if mask.size(0) != output.size(0):
          mask = mask.expand_as(output)
        output_masked = output[mask]
        target_masked = target[mask]
        weights_masked = weights[mask]
        # Ensure there is at least one non-NaN element to avoid empty tensor errors
        if output_masked.numel() == 0:
          # Return a zero loss if all elements are NaN, or handle this case appropriately
          return torch.tensor(0.0, device=output.device)


        # Calculate squared error element-wise
        elementwise_error = (output_masked - target_masked) ** 2
        # Apply weights to each squared error
        weighted_errors = elementwise_error * weights_masked
        # Sum all weighted errors and normalize by sum of weights (weighted mean)
        weighted_mean_error = torch.sum(weighted_errors) / torch.sum(weights_masked)
        return weighted_mean_error
#        return loss_fn_f(output_masked, target_masked) * weights_masked#, pearson_loss(output_masked, target_masked))



      embedders = [SentenceTransformer(e) for e in s.embedding]
      model = GoalTypeScoringModel(embedders, query=s.query, dropout_rate=s.dropout_rate,hidden=s.hidden_layers,activation=s.activation, attend=s.attention_heads, trainable_embeddings=s.trainable_embeddings, batch_norm=s.batch_norm)
      print(model)
      s.model = model
      if s.pretrain_epochs > 0:
        print("starting pretraining [finetuning of embedders]")
        pretrain_set = PretrainDataset(train_set.dataframe)
        pretrain_val_set = PretrainDataset(pd.concat((test_set.dataframe, train_set.dataframe[train_set.dataframe['is_prototype']])))

        pretrain_loader = DataLoader(pretrain_set, batch_size=s.pretrain_batch_size, shuffle=True)
        for i,e in tqdm(enumerate(embedders)):
          val_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
            [x for x in pretrain_val_set],
            batch_size=s.pretrain_batch_size,
            name=f'eval_embedding_{self.name}_{i}.csv',
            write_csv=True
            )
          train_loss = losses.CosineSimilarityLoss(e)
          e.old_fit(train_objectives=[(pretrain_loader, train_loss)],
                epochs=s.pretrain_epochs,
                warmup_steps=math.ceil(len(pretrain_set)*0.1*s.pretrain_epochs),
                evaluation_steps=int(len(pretrain_set)*0.1)#,
                #evaluator=val_evaluator
                )

      model.to(device)

      optimizer = s.optimizer(model.parameters(), lr=s.learning_rate)
      cosine_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=s.epochs//3)

      print('starting training loop')

      lowest_loss = 999999999999
      best_epoch = 0
      best_corr = None
      train_loader = DataLoader(train_set, batch_size=s.batch_size, shuffle=True)
      test_loader = DataLoader(test_set, batch_size=s.batch_size, shuffle=False)
      for epoch in tqdm(range(s.epochs)):  # Number of epochs
        try:
          model.train()
          for sentences, labels, weights in tqdm(train_loader):
            labels = labels.to(device)
            weights = weights.to(device).expand(-1, 3)
            optimizer.zero_grad()
            outputs = model(sentences)
            torch.cuda.synchronize()
            if torch.isnan(outputs).any() or torch.isinf(outputs).any():
              print("NaN or Inf in outputs")
              df = train_loader.dataset.dataframe
              df_new = df[df['item'].isin(sentences)]
              print(df_new)
              continue  # Skip this batch or handle the issue
            loss = loss_fn(outputs, labels, weights)
            torch.cuda.synchronize()
            loss.backward()
            optimizer.step()
            cosine_scheduler.step()
          print(f"Epoch {epoch+1}, Loss: {loss.item()}, LR: {cosine_scheduler.get_last_lr()}")
          test_loss,corr = evaluate_model(model, test_loader, loss_fn, device, 0.1)
          if best_corr is None:
            best_corr = corr
          print(f"Average Test Loss: {test_loss}")
          if test_loss < lowest_loss:
            print(f"*** New Best Model *** \n\t@Epoch: {epoch+1}\n\tTest Loss: {round_to_first_non_zero_digit(test_loss)}\n\timproved by: {round_to_first_non_zero_digit(lowest_loss - test_loss)} (down from {round_to_first_non_zero_digit(lowest_loss)} in epoch {best_epoch+1})")
            lowest_loss = test_loss
            best_epoch = epoch
            print("correlations:")
            print(corr)
            trend = corr[['corr_prog','corr_mant','corr_prot']] - best_corr[['corr_prog','corr_mant','corr_prot']]
            trend['source'] = corr['source']
            print("trend:")
            print(trend)
            best_corr = corr
            s.best_validation_loss = lowest_loss
            s.best_epoch = epoch+1

            best_val_loss = lowest_loss
            save_checkpoint({
                'epoch': epoch + 1,
                'state_dict': model.state_dict(),
                'best_val_loss': best_val_loss,
                'optimizer': optimizer.state_dict(),
            }, filename=f"{self.name}.{s_num}.pth.tar")
        except:
          print(f"error in epoch {epoch}")
          model.to("cpu")
          raise
      print("loading best checkpoint")
      checkpoint = torch.load(f'{self.name}.{s_num}.pth.tar', map_location='cpu')
      model.load_state_dict(checkpoint['state_dict'])
      optimizer.load_state_dict(checkpoint['optimizer'])
      model.to("cpu")

    print('finished training all models')




In [ ]:

def combined_loss(fn_a, fn_b, w_a=1, w_b=1):
  def c_loss(output, target):
    return (fn_a(output,target) * w_a) + (fn_b(output,target) * w_b)
  return c_loss

custom = combined_loss(F.binary_cross_entropy, F.mse_loss)

In [ ]:
import os
import glob

path = 'human_scores/'
extension = 'csv'
os.chdir(path)
small_files = glob.glob('*small.{}'.format(extension))
small_sets = {}
for f in small_files:
  small_sets[f] = [i for i in pd.read_csv(f).item]
os.chdir('..')

In [ ]:
#only_dutch = lambda x: x['source'] in ('BelgianHumanScores.csv', 'DutchNLHumanScores.csv')
#be_small = pd.read_csv('human_scores/BelgianNLHumanScores_small.csv')
#be_items = [i for i in be_small.item]
#nl_small = pd.read_csv('human_scores/DutchNLHumanScores_small.csv')
#nl_items = [i for i in nl_small.item]
names = {
    'AllTokensHumanScores2012.csv': 'HumanScores2012_small.csv',
    'DutchNLHumanScores.csv': 'DutchNLHumanScores_small.csv',
    'BelgianHumanScores.csv': 'BelgianNLHumanScores_small.csv',
    'GermanHumanScores2021.csv': 'HumanScoresGerman_small.csv',
}
def only_small_set(row):
  #skip dutch
  if row['source'] in ('BelgianHumanScores.csv', 'DutchNLHumanScores.csv'):
    return False
  if row['source'] in names.keys():
    return row['item'] in small_sets[names[row['source']]]
  return True  # Keep all other rows
train,test = read_file("paraphrased_data.csv", 0.75, filter=only_small_set)
import ast
train['extract'] = train['extract'].apply(ast.literal_eval)
train_set = TextDataset(train, add_prototypes=True, paraphrases_column='extract')
test_set = TextDataset(test)
train_set.dataframe  = train_set.dataframe.reset_index(drop=True)
test_set.dataframe = test_set.dataframe.reset_index(drop=True)
train_set.dataframe.to_csv(f"train_set_all_{timestr}.csv")
test_set.dataframe.to_csv(f"test_set_all_{timestr}.csv")
len(train_set.dataframe)

1262

In [ ]:
print(train_set.dataframe.source.unique(), small_files)

[nan 'AllTokensHumanScores2012.csv' 'GermanHumanScores2021.csv'
 'total_sample_manifesto_humanRatings.csv'
 'total_sample_speeches_humanRatings.csv'] ['BelgianNLHumanScores_small.csv', 'DutchNLHumanScores_small.csv', 'HumanScores2012_small.csv', 'HumanScoresGerman_small.csv']


In [ ]:
train_set.dataframe.source.unique()

array([nan, 'AllTokensHumanScores2012.csv', 'GermanHumanScores2021.csv',
       'total_sample_manifesto_humanRatings.csv',
       'total_sample_speeches_humanRatings.csv'], dtype=object)

In [ ]:
train=pd.read_csv("train_set_dutch_2024-05-20.csv")
test=pd.read_csv("test_set_dutch_2024-05-20.csv")
train_set = TextDataset(train)
test_set = TextDataset(test)
train

NameError: name 'TextDataset' is not defined

In [ ]:
query_a = "To what extend does this represent a progress goal, a maintenance goal, and a protection goal"
queries = ["We aim to invest in maintaining the current state", "We willen investeren in het behoud van de huidige staat", "Wir wollen in die Erhaltung des derzeitigen Zustands investieren", "We aim to invest in making changes and improving the current state", "We willen investeren in het aanbrengen van veranderingen en het verbeteren van de huidige staat", "Wir wollen in Veränderungen und die Verbesserung des aktuellen Zustands investieren", "We aim to invest in preventing negative changes to the current state", "We willen investeren in het voorkomen van negatieve veranderingen in de huidige staat","Wir wollen in die Verhinderung negativer Veränderungen des derzeitigen Zustands investieren"]
specs = [
    #ModelSpec(embedding=['BAAI/bge-m3'], hidden_layers=[], epochs=300, pretrain_epochs=2, batch_size=256, pretrain_batch_size=64, batch_norm=True),
    ModelSpec(embedding=['paraphrase-multilingual-mpnet-base-v2'], hidden_layers=[], epochs=100, pretrain_epochs=10, batch_size=256, pretrain_batch_size=128, batch_norm=True)
    #{'embedding': ['BAAI/bge-m3', 'intfloat/multilingual-e5-large'], 'hidden_layers': [], 'epochs' : 30, "batch_size" : 32, "batch_norm": True },
    #{'embedding': ['BAAI/bge-m3', 'intfloat/multilingual-e5-large'], 'hidden_layers': [1024, 512], 'epochs' : 30, "batch_size" : 32, "batch_norm": True },
    #{'embedding': ['BAAI/bge-m3', 'intfloat/multilingual-e5-large'], 'hidden_layers': [2048*2,2048*2,2048*2], 'epochs' : 120, "batch_size" : 128, "batch_norm": True, "attend":0, "query": queries },
    #{'embedding': ['NetherlandsForensicInstitute/robbert-2022-dutch-sentence-transformers'], 'learning_rate':0.0001 , 'hidden_layers': [2048*2,2048*2,2048*2], 'epochs' : 120, "batch_size" : 128, "batch_norm": True, "attend":0, "query": queries },
    #{'embedding': ['BAAI/bge-m3', 'intfloat/multilingual-e5-large'], 'hidden_layers': [512,256,128,64,32], 'epochs' : 400, "batch_size" : 128, "batch_norm": True },
         ]
models = GoalTypeScoringModelFactory(specs, name="no_hidden2_pretrain_10_clean_DE_EN")
#test_set.dataframe = test_set.dataframe.drop(columns=['prog_ai','mant_ai','prot_ai'])
models.train_all(train_set, test_set, "cuda")

training model for spec
	 ModelSpec(embedding=['paraphrase-multilingual-mpnet-base-v2'], dropout_rate=0.5, hidden_layers=[], activation=<class 'torch.nn.modules.activation.LeakyReLU'>, batch_size=256, optimizer=<class 'pytorch_ranger.ranger.Ranger'>, learning_rate=0.001, epochs=100, pretrain_epochs=10, pretrain_batch_size=128, attention_heads=0, trainable_embeddings=False, batch_norm=True, query=None, model=None, best_validation_loss=inf, best_epoch=-1)
GoalTypeScoringModel(
  (all): Sequential(
    (0): ParallelLayer(
      (elements): ModuleList(
        (0): EmbeddingEncoder(
          (embedder): SentenceTransformer(
            (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
            (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pool

0it [00:00, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/86 [00:00<?, ?it/s]

Iteration:   0%|          | 0/86 [00:00<?, ?it/s]

...

Epoch 98, Loss: 0.030508920550346375, LR: [5.558227567254392e-05]
Average Test Loss: 0.02792210876941681


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 99, Loss: 0.024417191743850708, LR: [0.0]
Average Test Loss: 0.027934979647397995


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 100, Loss: 0.03436168655753136, LR: [5.55822756725431e-05]
Average Test Loss: 0.027931636199355125
loading best checkpoint
finished training all models


In [ ]:
torch.save(specs[0].model, 'model__no_hidden2_pretrain_10_all_clean_DE_EN.pt') # Export to TorchScript


In [ ]:
model = specs[0].model

In [ ]:
import os
import glob

path = 'datafiles'
extension = 'csv'
os.chdir(path)
files = glob.glob('*.{}'.format(extension))
print(files)

['2008platform_tokens.csv', 'allTokens_BE.csv', 'AllTokensHumanScores2012.csv', 'AllTokens_NL.csv', 'AllTokensGerman2021.csv']


In [ ]:
df = pd.read_csv(files[0])


In [ ]:
dl = DataLoader(df.item)

In [ ]:
model.to("cuda")
model.eval()
for f in files:
  df = pd.read_csv(f)
  df['model_ratings'] = df.apply(lambda x: model(x['item']).cpu().detach().numpy(), axis=1)
  df['prot_model'] = df.apply(lambda x: x.model_ratings[0], axis=1)
  df['mant_model'] = df.apply(lambda x: x.model_ratings[1], axis=1)
  df['prog_model'] = df.apply(lambda x: x.model_ratings[2], axis=1)
  df.drop('model_ratings', axis='columns')
  df.to_csv(f'rated/all_clean_rated_DE_EN{f}')
model.to("cpu")

GoalTypeScoringModel(
  (all): Sequential(
    (0): ParallelLayer(
      (elements): ModuleList(
        (0): EmbeddingEncoder(
          (embedder): SentenceTransformer(
            (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
            (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
          )
        )
      )
    )
    (1): Dropout(p=0.5, inplace=False)
    (2): Linear(in_features=768, out_features=3, bias=True)
  )
)

In [ ]:
df

,number,year,party,agenda,item,model_ratings,prot_model
0,17,2008,Republicans,Reforming Government,Constrain the federal government to its legiti...,"[0.63545066, 0.49470675, 0.71112084]",0.635451
1,18,2008,Republicans,Reforming Government,"With so many redundant, inefficient, and ineff...","[0.7030175, 0.45272344, 0.6537735]",0.703017
2,19,2008,Republicans,Reforming Government,"As the Declaration of Independence states, our...","[0.55710304, 0.6539965, 0.53333104]",0.557103
3,20,2008,Republicans,Reforming Government,Because official Washington does not even know...,"[0.6640916, 0.5932683, 0.5353352]",0.664092
4,21,2008,Republicans,Reforming Government,The expected retirement over the next ten year...,"[0.7240357, 0.5326127, 0.57005686]",0.724036
...,...,...,...,...,...,...,...
187,306,2008,Democrats,Renewing American Democracy,We support campaign finance reform to reduce t...,"[0.7408665, 0.4197374, 0.6125624]",0.740866
188,307,2008,Democrats,Renewing American Democracy,"For our Judiciary, we will select and confirm ...","[0.7361378, 0.47045344, 0.62628686]",0.736138
189,308,2008,Democrats,Renewing American Democracy,We support the appointment of judges who respe...,"[0.6634211, 0.54448557, 0.6333358]",0.663421
190,309,2008,Democrats,Renewing American Democracy,We will work to fully protect and enforce the ...,"[0.67700523, 0.63648456, 0.60427487]",0.677005


In [ ]:
from google.colab import runtime
runtime.unassign()